In [1]:
import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN
from sklearn.metrics import classification_report
import xgboost as xgb

from torch import nn, optim
import torch.nn.functional as F

## 1)

In [2]:
# read dataset

x_dataset = pd.read_pickle("./x_dataset.pkl")
y_dataset = pd.read_pickle("./y_dataset.pkl")


In [3]:
# del unused columns

del x_dataset["id"]
del y_dataset["user_id"]

In [4]:
y_dataset = y_dataset.astype('float64')

In [5]:
y_dataset.dtypes

ae264e3637204a6fb9bb56bc8210ddfd    float64
4d5c57ea9a6940dd891ad53e9dbe8da0    float64
3f207df678b143eea3cee63160fa8bed    float64
9b98b8c7a33c4b65b9aebfe6a799e6d9    float64
0b1e1539f2cc45b7b9fa7c272da2e1d7    float64
2298d6c36e964ae4a3e7e9706d1fb8c2    float64
fafdcd668e3743c1bb461111dcafc2a4    float64
5a8bc65990b245e5a138643cd4eb9837    float64
f19421c1d4aa40978ebb69ca19b0e20d    float64
2906b810c7d4411798c6938adc9daaa5    float64
dtype: object

In [6]:
x_dataset.head()

,age,income,user_day,gender_F,gender_M,gender_O
1,0.445783,0.911111,0.206253,1,0,0
3,0.686747,0.777778,0.243006,1,0,0
5,0.602410,0.444444,0.049918,0,1,0
8,0.566265,0.255556,0.091607,0,1,0
12,0.481928,0.233333,0.140976,0,1,0


In [12]:
xTrain, xTest, yTrain, yTest = train_test_split(x_dataset, y_dataset, test_size = 0.2, random_state = 0)

NameError: name 'x_dataset' is not defined

In [8]:
# if GPUs are available they are used

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
# Clasifier with pytorch with n input features, 2 hidden layers and an output features

class Classifier(nn.Module):
    def __init__(self, input_features, hidden_dim_1, hidden_dim_2, output_dim):
        """
        Initialize the model by setting up linear layers.
        Use the input parameters to help define the layers of your model.
        :param input_features: the number of input features in your training/test data
        :param hidden_dim: helps define the number of nodes in the hidden layer(s)
        :param output_dim: the number of outputs you want to produce
        """
        super().__init__()
        
        #Fully conected
        self.fc1 = nn.Linear(input_features, hidden_dim_1)
        self.fc2 = nn.Linear(hidden_dim_1, hidden_dim_2)
        self.fc3 = nn.Linear(hidden_dim_2, output_dim)
       
         # Droput 
        self.drop = nn.Dropout(0.3)
        
        # Softmax Layer
        self.sig = nn.Sigmoid()
        

# The feed forward is with relu activation on layers, 0.3 of droput and a sigmoid activation

    def forward(self, x):
        """
        Perform a forward pass of our model on input features, x.
        :param x: A batch of input features of size (batch_size, input_features)
        :return: A single, sigmoid-activated value as output
        """
        
        # feedforward behavior
        
        out = F.relu(self.fc1(x))
        out = self.drop(out)
        out = F.relu(self.fc2(out))
        out = self.drop(out)
        out = self.fc3(out)
        
        return self.sig(out)
        

In [85]:
# This model has 6 inputs 2 hidden layers and 10 oputputs as there is 10 different offers

model = Classifier(xTrain.shape[1], 16, 32, yTrain.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [86]:
%%time

epochs = 1
model = model.train()
model = model.double()
for e in range(epochs):
    running_loss = 0
    for x, y in zip(xTrain.values, yTrain.values):
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        log_ps = model(x)

        loss = criterion(log_ps, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(xTrain)}")

Training loss: 0.3033990758627614
CPU times: user 8.3 s, sys: 644 ms, total: 8.95 s
Wall time: 9.04 s


In [16]:
model.eval()
running_loss = 0
for x, y in zip(xTest.values, yTest.values):
    x = torch.from_numpy(x).to(device)
    y = torch.from_numpy(y).to(device)
    log_ps = model(x)

    loss = criterion(log_ps, y)

    optimizer.zero_grad()
    running_loss += loss.item()
else:
    print(f"Test loss: {running_loss/len(xTest)}")

Test loss: 0.3696391788351165


In [3]:
!pip install scikit-multilearn --user

    100% |████████████████████████████████| 92kB 3.5MB/s ta 0:00:011


## 1.1)

In [37]:

y_dataset = pd.read_pickle("./y_dataset1.pkl")

In [38]:
del y_dataset["user_id"]

In [39]:
y_dataset = y_dataset.astype('float64')

In [40]:
x_dataset.head()

,age,income,user_day,gender_F,gender_M,gender_O
1,0.445783,0.911111,0.206253,1,0,0
3,0.686747,0.777778,0.243006,1,0,0
5,0.602410,0.444444,0.049918,0,1,0
8,0.566265,0.255556,0.091607,0,1,0
12,0.481928,0.233333,0.140976,0,1,0


In [41]:
xTrain, xTest, yTrain, yTest = train_test_split(x_dataset, y_dataset, test_size = 0.2, random_state = 0)

Different aprroaches using a combination of gaussian naive bayes and different clasifiers 

In [42]:

# initialize binary relevance multi-label classifier
# with a gaussian naive bayes base classifier

classifier = BinaryRelevance(GaussianNB())

# train
classifier.fit(xTrain, yTrain)

# predict
predictions = classifier.predict(xTest)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(yTest,predictions)

0.31062394603709947

In [45]:
# using classifier chains

# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(GaussianNB())

# train
classifier.fit(xTrain.values, yTrain.values)

# predict
predictions = classifier.predict(xTest)

accuracy_score(yTest,predictions)

0.31062394603709947

In [46]:
# using Label Powerset

# initialize Label Powerset multi-label classifier
# with a gaussian naive bayes base classifier
classifier = LabelPowerset(GaussianNB())

# train
classifier.fit(xTrain.values, yTrain.values)

# predict
predictions = classifier.predict(xTest)

accuracy_score(yTest.values,predictions)

0.00033726812816188871

In [48]:
%%time

classifier = MLkNN(k=20)

# train
classifier.fit(xTrain.values, yTrain.values)

# predict
predictions = classifier.predict(xTest)

print(accuracy_score(yTest,predictions))

0.31197301855
CPU times: user 32.6 s, sys: 0 ns, total: 32.6 s
Wall time: 32.8 s


## 2) 

In [29]:
x_dataset2 = pd.read_pickle("./x_dataset.pkl")
y_dataset2 = pd.read_pickle("./y_dataset2.pkl")


In [30]:
del x_dataset2["id"]
del y_dataset2["user_id"]

In [31]:
y_dataset2 = y_dataset2.astype('category')
y_dataset2 = y_dataset2.astype('int32')

In [32]:
xTrain2, xTest2, yTrain2, yTest2 = train_test_split(x_dataset2, y_dataset2, test_size = 0.2, random_state = 0)

This model is similar to the one before but in this case there is only 1 class on the output

In [94]:
model2 = Classifier(xTrain2.shape[1], 16, 32, yTrain2.shape[1]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [96]:
%%time

epochs = 15
model2 = model2.double()
for e in range(epochs):
    running_loss = 0
    for x, y in zip(xTrain2.values, yTrain2.values):
        x = torch.from_numpy(x).to(device)
        y = torch.from_numpy(y).to(device)
        log_ps = model2(x)

        loss = criterion(log_ps, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(xTrain2)}")

Training loss: 0.6793762536120178
Training loss: 0.6739288186936636
Training loss: 0.6772462723809061
Training loss: 0.6819372286704318
Training loss: 0.6851573443875949
Training loss: 0.6785401645326762
Training loss: 0.6811660978095787
Training loss: 0.6828188711823258
Training loss: 0.6813273380707796
Training loss: 0.6725183949447788
Training loss: 0.6817451784378211
Training loss: 0.6843612651145882
Training loss: 0.683511632155457
Training loss: 0.6814843291514658
Training loss: 0.6744948160259057
CPU times: user 2min 7s, sys: 10.2 s, total: 2min 17s
Wall time: 2min 18s


Trying an xgboos clasification to clasify only 1 label (the top rewarded)

In [33]:

dtrain = xgb.DMatrix(data=xTrain2, label=yTrain2.values)
dtest = xgb.DMatrix(data=xTest2)

In [34]:
params = {
    'objective': 'multi:softmax',  # error evaluation for multiclass training
    'num_class': 10,
    'n_gpus': 0
}

In [35]:
bst = xgb.train(params, dtrain)
pred = bst.predict(dtest)

from sklearn.metrics import classification_report

print(classification_report(yTest2, pred))

             precision    recall  f1-score   support

          0       0.41      0.97      0.58      1209
          1       0.20      0.02      0.04       423
          3       0.17      0.01      0.01       198
          4       0.00      0.00      0.00       109
          5       0.12      0.01      0.02       313
          6       0.26      0.02      0.03       275
          8       0.38      0.01      0.02       349
          9       0.00      0.00      0.00        89

avg / total       0.29      0.40      0.25      2965



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
